In [1]:
import numpy as np
import open3d as o3d  # type: ignore

from superprimitive_fusion.mesh_fusion_utils import (
    smooth_normals,
    calc_local_spacing,
    compute_overlap_set_cached,
    smooth_overlap_set_cached,
    precompute_cyl_neighbours,
    trilateral_shift_cached,
    find_boundary_edges,
    topological_trim,
    merge_nearby_clusters,
)
from superprimitive_fusion.scanner import (
    virtual_rgbd_scan
)
from superprimitive_fusion.utils import (
    bake_uv_to_vertex_colours,
    polar2cartesian,
)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
mesh = o3d.io.read_triangle_mesh("../data/mustard-bottle/textured.obj", enable_post_processing=True)

bake_uv_to_vertex_colours(mesh)

mesh.compute_vertex_normals()

bb = mesh.get_minimal_oriented_bounding_box()
scale = np.mean(bb.get_max_bound())

In [3]:
scans = []
for i in range(2):
    mesh_scan, pcd_scan = virtual_rgbd_scan(
        mesh,
        cam_centre=polar2cartesian(r=0.3, lat=60, long=0+60*i),
        look_dir=(0, 0, 0),
        width_px=180,
        height_px=120,
        fov=70,
        dropout_rate=0,
        depth_error_std=0.002*scale,
        translation_error_std=0,#0.02*scale,
        rotation_error_std_degs=0,
        dist_thresh=0.25*scale,
    )
    scans.append(mesh_scan)

o3d.visualization.draw_geometries(
    scans,
    window_name="Virtual scan",
    front=[0.3, 1, 0],
    lookat=[0, 0, 0],
    up=[0, 0, 1],
    zoom=0.7,
)

In [4]:
# mesh1 = o3d.io.read_triangle_mesh("meshes/bottle_1.ply")
# mesh2 = o3d.io.read_triangle_mesh("meshes/bottle_2.ply")
mesh1 = scans[0]
mesh2 = scans[1]
h_alpha: float = 3.0
r_alpha: float = 2.0
trilat_iters: int = 2
nrm_smth_iters: int = 1
shift_all: bool = False

In [5]:
# ---------------------------------------------------------------------
# Raw geometry & attribute extraction
# ---------------------------------------------------------------------
points1 = np.asarray(mesh1.vertices)
points2 = np.asarray(mesh2.vertices)

pointclouds = (points1, points2)
points = np.vstack(pointclouds)

colours1 = mesh1.vertex_colors
colours2 = mesh2.vertex_colors
colours = np.concatenate([colours1, colours2], axis=0)

kd_tree = o3d.geometry.KDTreeFlann(points.T)

In [6]:
# ---------------------------------------------------------------------
# Normals
# ---------------------------------------------------------------------
mesh1.compute_vertex_normals()
mesh2.compute_vertex_normals()

normals1 = np.asarray(mesh1.vertex_normals)
normals2 = np.asarray(mesh2.vertex_normals)
normals = np.concatenate([normals1, normals2], axis=0)

scan_ids = np.concatenate([np.full(len(pts), i) for i, pts in enumerate(pointclouds)])

normals = smooth_normals(points, normals, tree=kd_tree, k=8, T=0.7, n_iters=nrm_smth_iters)

In [7]:
# ---------------------------------------------------------------------
# Local geometric properties
# ---------------------------------------------------------------------
local_spacing_1, local_density_1 = calc_local_spacing(points1, points1, tree=kd_tree)
local_spacing_2, local_density_2 = calc_local_spacing(points2, points2, tree=kd_tree)
local_spacings = (local_spacing_1, local_spacing_2)

local_spacing = np.concatenate(local_spacings)
local_density = np.concatenate((local_density_1, local_density_2))

global_avg_spacing = (1 / len(local_spacings)) * np.sum(
    [(1 / len(ls)) * np.sum(ls) for ls in local_spacings]
)

In [8]:
# ---------------------------------------------------------------------
# Overlap detection
# ---------------------------------------------------------------------
nbr_cache = precompute_cyl_neighbours(points, normals, local_spacing, r_alpha, h_alpha, kd_tree)

overlap_idx, overlap_mask = compute_overlap_set_cached(scan_ids, nbr_cache)
overlap_idx, overlap_mask = smooth_overlap_set_cached(overlap_mask, nbr_cache)
overlap_idx, overlap_mask = smooth_overlap_set_cached(overlap_mask, nbr_cache, p_thresh=1)

In [9]:
debug_colours = np.array([
    [255,   0,   0],
    [  0, 255,   0],
])
f0 = overlap_mask
f1 = ~overlap_mask
filters = np.column_stack((f0, f1))
idx = filters.argmax(axis=1)
row_colours = debug_colours[idx]
# row_colours[oidx] = np.array([0,0,0])

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(row_colours)

# o3d.visualization.draw_geometries([pcd])

In [10]:
m1 = scan_ids==0
m2 = overlap_mask
debug_colours = np.array([
    [255,   0,   0],
    [  0, 255,   0],
    [  0,   0, 255],
    [255,   0, 255],
])
f0 = ~m1 & ~m2
f1 =  m1 & ~m2
f2 = ~m1 &  m2
f3 =  m1 &  m2
filters = np.column_stack((f0, f1, f2, f3))
idx = filters.argmax(axis=1)
row_colours = debug_colours[idx]

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(row_colours)

# o3d.visualization.draw_geometries([pcd])

In [11]:
# ---------------------------------------------------------------------
# Find overlap boundary edges
# ---------------------------------------------------------------------
tris1 = np.asarray(mesh1.triangles)
tris2 = np.asarray(mesh2.triangles) + len(points1)  # shift indices
all_tris = np.concatenate([tris1, tris2], axis=0)

nonoverlap_tris = all_tris[~np.all(overlap_mask[all_tris], axis=1)]
boundary_edges = find_boundary_edges(nonoverlap_tris)

In [12]:
# ---------------------------------------------------------------------
# Trilateral point shifting
# ---------------------------------------------------------------------
trilat_shifted_pts = points.copy()
for _ in range(trilat_iters):
    trilat_shifted_pts = trilateral_shift_cached(trilat_shifted_pts, normals, local_spacing, local_density, overlap_idx, nbr_cache, r_alpha, h_alpha, shift_all)

kd_tree = o3d.geometry.KDTreeFlann(trilat_shifted_pts.T)

In [13]:
# ---------------------------------------------------------------------
# Merge nearby clusters
# ---------------------------------------------------------------------
cluster_mapping, clustered_overlap_pnts, clustered_overlap_cols, clustered_overlap_nrms = merge_nearby_clusters(
    trilat_shifted_pts=trilat_shifted_pts,
    normals=normals,
    colours=colours,
    overlap_mask=overlap_mask,
    overlap_idx=overlap_idx,
    global_avg_spacing=global_avg_spacing,
    h_alpha=h_alpha,
    tree=kd_tree,
)

In [14]:
# ---------------------------------------------------------------------
# Classify vertices
# ---------------------------------------------------------------------
tri_has_overlap_any = overlap_mask[all_tris].any(axis=1)
overlap_any_idx = np.unique(all_tris[tri_has_overlap_any])

border_mask = np.zeros(len(points), dtype=bool)
border_mask[overlap_any_idx] = True
border_mask[cluster_mapping != -1] = False

nonoverlap_nonborder_mask = np.zeros(len(points), dtype=bool)
nonoverlap_nonborder_mask[cluster_mapping == -1] = True
nonoverlap_nonborder_mask[border_mask] = False

n_overlap = len(clustered_overlap_pnts)
n_border = border_mask.sum()
n_free = nonoverlap_nonborder_mask.sum()

new_points = np.concatenate(
    [
        clustered_overlap_pnts,
        trilat_shifted_pts[border_mask],
        trilat_shifted_pts[nonoverlap_nonborder_mask],
    ],
    axis=0,
)
new_colours = np.concatenate(
    [
        clustered_overlap_cols,
        colours[border_mask],
        colours[nonoverlap_nonborder_mask],
    ],
    axis=0,
)
new_normals = np.concatenate(
    [
        clustered_overlap_nrms,
        normals[border_mask],
        normals[nonoverlap_nonborder_mask],
    ],
    axis=0,
)

new_colours = np.clip(new_colours, 0, 1)

In [15]:
# ---------------------------------------------------------------------
# Complete mapping
# ---------------------------------------------------------------------
border_idx_from = np.arange(len(points))[border_mask]
border_idx_to = np.arange(n_border) + n_overlap

free_idx_from = np.arange(len(points))[nonoverlap_nonborder_mask]
free_idx_to = np.arange(n_free) + n_overlap + n_border

mapping = cluster_mapping.copy()
mapping[border_idx_from] = border_idx_to
mapping[free_idx_from] = free_idx_to

In [16]:
# ---------------------------------------------------------------------
# Mesh the overlap zone
# ---------------------------------------------------------------------
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(new_points[:n_overlap])

if new_normals[:n_overlap] is not None:
    pcd.normals = o3d.utility.Vector3dVector(new_normals[:n_overlap])
else:
    pcd.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(
            radius=2.5 * global_avg_spacing, max_nn=30
        )
    )
    pcd.orient_normals_consistent_tangent_plane(k=30)

r_min = global_avg_spacing
radii = o3d.utility.DoubleVector(np.geomspace(r_min, r_min*4, num=5))

overlap_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, radii
)

overlap_mesh.remove_duplicated_vertices()
overlap_mesh.remove_duplicated_triangles()
overlap_mesh.remove_degenerate_triangles()
overlap_mesh.remove_non_manifold_edges()
overlap_mesh.compute_vertex_normals()

TriangleMesh with 2657 points and 5165 triangles.

In [17]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(new_points)
colours = np.concatenate([
    np.full((n_overlap,3),[255, 0, 0]),
    np.full((n_border,3), [0, 255, 0]),
    np.full((n_free,3),   [0, 0, 255]),
])
pcd.colors = o3d.utility.Vector3dVector(colours)

# o3d.visualization.draw_geometries([overlap_mesh, pcd])

In [18]:
new_nonoverlap_tris = mapping[nonoverlap_tris]
nonoverlap_mesh = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(new_points),
    triangles=o3d.utility.Vector3iVector(new_nonoverlap_tris),
)

# o3d.visualization.draw_geometries([nonoverlap_mesh, pcd])

In [19]:
# o3d.visualization.draw_geometries([mesh1, mesh2, pcd])

In [19]:
# ---------------------------------------------------------------------
# Trim overlap mesh
# ---------------------------------------------------------------------
mapped_boundary_edges = mapping[boundary_edges]
relevant_boundary_edges = mapped_boundary_edges[
    np.all(mapped_boundary_edges < len(overlap_mesh.vertices), axis=1)
]

trimmed_overlap_mesh = topological_trim(
    overlap_mesh, relevant_boundary_edges
)

# o3d.visualization.draw_geometries([trimmed_overlap_mesh, pcd])

In [20]:
trimmed_overlap_tris = np.asarray(trimmed_overlap_mesh.triangles)
fused_mesh_triangles = np.concatenate(
    [trimmed_overlap_tris, mapping[nonoverlap_tris]], axis=0
)

# fused_mesh = o3d.geometry.TriangleMesh(
#     vertices=o3d.utility.Vector3dVector(new_points),
#     triangles=o3d.utility.Vector3iVector(fused_mesh_triangles),
# )

# fused_mesh.vertex_colors = o3d.utility.Vector3dVector(new_colours)

# fused_mesh.remove_unreferenced_vertices()
# fused_mesh.remove_duplicated_triangles()
# fused_mesh.remove_duplicated_vertices()
# fused_mesh.remove_degenerate_triangles()
# fused_mesh.remove_non_manifold_edges()

# o3d.visualization.draw_geometries([fused_mesh, pcd])

In [21]:
import pymeshfix

mf = pymeshfix.PyTMesh()
mf.load_array(new_points, fused_mesh_triangles)
mf.fill_small_boundaries(nbe=10)

points, faces = mf.return_arrays()

repaired = o3d.geometry.TriangleMesh()
repaired.vertices = o3d.utility.Vector3dVector(points)
repaired.triangles = o3d.utility.Vector3iVector(faces)
repaired.vertex_colors = o3d.utility.Vector3dVector(new_colours)

o3d.visualization.draw_geometries([repaired])

: 